Virtual File Systems a la linux /proc and /sys
to control cburnfs

    /@ - cosmos root
    /@/hosts - host configuration vfs
    


In [1]:
from fs.osfs import OSFS
from fs.info import Info
from os import statvfs

In [57]:

class OSFS_vfs(OSFS):
    def __init__(self, addr, **kwdargs):
        super().__init__(addr, **kwdargs)
        self.__addr = addr
        
    def getinfo(self, path='/',namespaces=None):
        info_raw = super().getinfo(path, namespaces).raw
        if namespaces != None and 'limits' in namespaces:
            st = statvfs(self.__addr)
            if st != None:
                bsize = st.f_bsize
                total = bsize * st.f_blocks
                used = total - (bsize * st.f_bfree)
                free = total - used
                info_raw['limits'] = {
                    'total': total,
                    'used': used,
                    'free': free
                }
        return Info(info_raw)
        

In [58]:
a = OSFS_vfs('demo-files/fs2')

In [59]:
info = a.getinfo('/', namespaces=['limits']).raw

In [61]:
print(info['limits']['free'] / 1024 / 1024 / 1024)

20.624351501464844


In [12]:
HostVFSTree = {
    'redhost': 'https://redhost.example.com',
    'reduser': 'UserVFSTree OVERRIDDEN!'
}

class HostVFS:
    def show_thyself(self):
        return self.myfsname   # reference to base class MyFS 
    
    def readtext(self,path):
        if path in HostVFSTree:
            return HostVFSTree[path]
        return super().readtext(path)
    
    def get_hostvfstree(self):
        return HostVFSTree
    

In [13]:
UserVFSTree = {
    'reduser': 'Raygan Henley'
}
class UserVFS:
    def readtext(self,path):
        if path in UserVFSTree:
            return UserVFSTree[path]
        return super().readtext(path)

In [14]:
class BaseFS:
    def readtext(self,path):
        return path[1:2]

In [15]:
class MyFS(UserVFS, HostVFS, BaseFS):
    def __init__(self):
        self.myfsname = 'royal'

In [16]:
b = MyFS()
b.readtext('reduser')

'Raygan Henley'

In [17]:
b.get_hostvfstree()

{'redhost': 'https://redhost.example.com',
 'reduser': 'UserVFSTree OVERRIDDEN!'}

In [18]:
b.show_thyself()

'royal'

In [55]:
def human_readable(size):
    for unit in ['B','k','M','G']:
        if abs(size) < 1024.0:
            return "%.1f%s" % (size, unit)
        size /= 1024.0
    return "%.0f%s" %(size, 'Yi')


In [56]:
human_readable(5*1024*2000)

'9.8M'